# 효율적인 정리를 위한 class화

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import numpy as np

class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
    def _build_net(self):
        with tf.variable_scope(self.name):
            learning_rate = 0.01
            keep_prob = 0.7
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            # Convolution layer1
            W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
            L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding="SAME")
            L1 = tf.nn.relu(L1)
            L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides = [1,2,2,1], padding="SAME")
            L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
            # Convolution layer2
            W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
            L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding="SAME")
            L2 = tf.nn.relu(L2)
            L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides = [1,2,2,1], padding="SAME")
            L2 = tf.nn.dropout(L2, keep_prob=keep_prob)
            # Convolution layer3
            W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev=0.01))
            L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding="SAME")
            L3 = tf.nn.relu(L3)
            L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides = [1,2,2,1], padding="SAME")
            L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
            L3 = tf.reshape(L3, [-1, 128 * 4 * 4])
            #FC layer
            W4 = tf.get_variable("W4", shape=[128 * 4 * 4, 625], initializer= tf.compat.v1.keras.initializers.glorot_normal)
            b4 = tf.Variable(tf.random_normal([625]))
            L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
            L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
            W5 = tf.get_variable("W5", shape=[625, 10], initializer=tf.compat.v1.keras.initializers.glorot_normal)
            b5 = tf.Variable(tf.random_normal([10]))
            hypothesis = tf.matmul(L4, W5) + b5
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=self.Y))
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
            correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(self.Y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#     def predict(self, x_test, keep_prop=1.0):
#         return self.sess.run(self.logits, feed_dict = {self.X: x_test,  keep_prob: keep_prop})
    
    def get_accuracy(self, x_test, y_test, keep_prop=1.0):
        (X_test, y_test), (X_test, y_test) = mnist.load_data()
        input_shape = X_test.shape[1] * X_test.shape[2]
        nb_classes = 10
        X_test = X_test.reshape(-1, input_shape) 
        y_test = to_categorical(y_test, nb_classes)
        return self.sess.run(self.accuracy, feed_dict={self.X: X_test, self.Y: y_test,  keep_prob: keep_prop})
    
    def train(self, X_data, y_data, keep_prob=0.7):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_data, self.Y: y_data})

C:\Users\user\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.wcdjnk7yvmpzq2me2zzhjjrj3jikndb7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
nb_classes = 10

In [3]:
input_shape = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(-1, input_shape) 
y_train = to_categorical(y_train, nb_classes)
X_train = tf.image.convert_image_dtype(X_train, tf.float32)
y_train = tf.image.convert_image_dtype(y_train, tf.float32)

In [4]:
sess = tf.InteractiveSession()
m1 = Model(sess, "m1")
sess.run(tf.global_variables_initializer())
print("Learning strated!")

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Learning strated!


In [5]:
training_epochs = 5
batch_size = 100

# 일반 epoch 구조로 모델 돌리기

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(60000/ batch_size)
    for i in tqdm(range(total_batch)):
        batch_xs, batch_ys = X_train[(i*batch_size):batch_size+(i*batch_size)].eval(), y_train[(i*batch_size):batch_size+(i*batch_size)].eval()
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / total_batch
    print("Epoch:", "%04d" % (epoch + 1), "cost =", "{:.9f}".format(avg_cost))
print("Leraning Finished!")
sess.close()

100%|██████████| 600/600 [02:59<00:00,  3.33it/s]


Epoch: 0001 cost = 0.586594294


100%|██████████| 600/600 [04:02<00:00,  2.48it/s]


Epoch: 0002 cost = 0.337489042


100%|██████████| 600/600 [04:56<00:00,  2.03it/s]


Epoch: 0003 cost = 0.309279121


 68%|██████▊   | 410/600 [03:59<01:57,  1.62it/s]

# ensemble 구조로 model 돌리기
- num_models: model를 여러번 학습하는 개수

In [ ]:
sess = tf.InteractiveSession()
models = []
num_models = 7
for m in range(num_models):
    models.append(Model(sess, "model"+ str(m)))
sess.run(tf.global_variables_initializer())
print("Learning Started!")

In [ ]:
for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(60000/ batch_size)
    for i in tqdm(range(total_batch)):
        batch_xs, batch_ys = X_train[(i*batch_size):batch_size+(i*batch_size)].eval(), y_train[(i*batch_size):batch_size+(i*batch_size)].eval()
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch
    print("Epoch:", "%04d"%(epoch + 1), 'cost =', avg_cost_list)
print("Learning Finished!")
sess.close()